In [9]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
# 클래스 정보
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/data/ephemeral/home/Lv2.Object_Detection/baseline/mmdetection/configs/_swin-l_atss/swin-l_atss.py')

# fold 번호에 따라 경로 변경
fold_number = 0  # 현재 fold 번호를 사용
root = f'/data/ephemeral/home/Lv2.Object_Detection/test_dir/{fold_number}/'

# dataset 경로 설정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root  # 수정된 부분: train/ 제거
cfg.data.train.ann_file = root + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)  # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)  # Resize 설정 추가

# (만약 test set을 실제로 따로 두었다면, test set도 설정)
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)  # Resize

# 학습 설정
cfg.data.samples_per_gpu = 4
cfg.seed = 2022
cfg.gpu_ids = [0]

# 모델 및 기타 설정
cfg.work_dir = './work_dirs/swin-l_atss(kfold)'
cfg.model.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()


In [12]:
import os
import wandb
from detectron2.engine import DefaultTrainer

# wandb 설정
wandb.init(project="mmdetectionA-project", config=cfg, name="ATSS_TEST")

# 필요한 경우 cfg 값 로그
wandb.config.update(cfg)

In [13]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [14]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [15]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-11 10:42:24,961 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth
2024-10-11 10:42:25,788 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-11 10:42:25,825 - mmdet - INFO - initialize ATSSHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'atss_cls', 'std': 0.01, 'bias_prob': 0.01}}
2024-10-11 10:42:25,827 - mmdet - INFO - 
backbone.patch_embed.projection.weight - torch.Size([192, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2024-10-11 10:42:25,829 - mmdet - INFO - 
backbone.patch_embed.projection.bias - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2024-10-11 10:42:25,829 - mmdet - INFO - 
backbone.patch_embed.norm.weight - torch.Size([192]): 
Initialized by user-defined `init

In [16]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-11 10:42:26,606 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-11 10:42:26,687 - mmdet - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/dyhead/atss_swin-l-p4-w12_fpn_dyhead_mstrain_2x_coco/atss_swin-l-p4-w12_fpn_dyhead_mstrain_2x_coco_20220509_100315-bc5b6516.pth


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2024-10-11 10:42:27,187 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for neck.0.lateral_convs.0.conv.weight: copying a param with shape torch.Size([256, 384, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 192, 1, 1]).
size mismatch for neck.0.lateral_convs.1.conv.weight: copying a param with shape torch.Size([256, 768, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 384, 1, 1]).
size mismatch for neck.0.lateral_convs.2.conv.weight: copying a param with shape torch.Size([256, 1536, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 768, 1, 1]).
size mismatch for bbox_head.atss_cls.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 256, 1, 1]).
size mismatch for bbox_head.atss_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
missing keys

KeyboardInterrupt: 

In [ ]:
# 학습 종료 시 로그
wandb.finish()